<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F3D8F5;">

# What is the inhibitory connectivity underlying surround suppression in the mouse primary visual cortex?

## Background
Surround suppression is a feature of sensory processing which, in visual systems, facilitates the definition of objects against their backgrounds (Kiniklioglu & Kaiser, 2025) by sharpening receptive fields as suppression strength increases (Plomp et al., 2019). While much work has been directed to describing the functional aspect of surround suppression, the underlying inhibitory structural connectivity has received less attention. We propose the use of the V1DD dataset, which comprises both structural and functional data, to address several questions about the inhibitory connectivity subserving surround suppression across the layers of the mouse primary visual cortex. 
    
## Methods
### Feature extraction:
Morphological metrics: total axonal length/radius, branch counts, maximum branch order, output synapse counts, synaptic density per unit length, number of distinct target types, layer/radial distributions, soma depth
Activity metrics: Differences in ΔF/F for windowed gabor - full-field gabor, suppression index
Synapse spatial targeting: Comparison of SOM+ vs other inhibitory interneurons for probability of synapsing
    
## Statistics:
Make a null connectivity model (by shuffling connections between cells): If the cell is connected to the inhibitory cells, then this cell may be more functionally capable of doing surround suppression, and the cells that could have been connected to inhibitory cells, will be less capable. Whether this is actually the case is unclear, and making a null connectivity model will help with this question.
Spearman correlation for cell connectivity probabilities? Maybe could do paired tests, depends on the final form of data.


In [1]:
# Import packages
import sys
import os
#import opencv
from os.path import join as pjoin
import platform
import glob

from caveclient import CAVEclient
import skeleton_plot as skelplot

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.sparse import csr_array
from typing import Union, Optional

from hdmf_zarr import NWBZarrIO
from nwbwidgets import nwb2widget

#from standard_transform.datasets import v1dd_ds

# Add the directory for the data and utilities
mat_version = 1196

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_root = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2025/"

# Set the directory to load prepared data 
#data_dir = '/data/'
mouse_id = '409828'
data_dir = glob.glob((os.path.join(data_root, mouse_id+'*')))[0]
data_dir #.head(5)

# and utility code
utils_dir = pjoin("..", "utils")

# Add utilities to path
sys.path.append(utils_dir)
from data_io import *
from utils import filter_synapse_table, check_index, adjacencyplot
from data_io import _get_data_dir

In [12]:
data_dir

'/data/409828_V1DD_GoldenMouse'

In [2]:
# Look at its metadata, sorted by "subject id' and 'session_number'
metadata = pd.read_csv(os.path.join(data_root, 'metadata', 'V1DD_metadata.csv'))
metadata.head()

,project_name,_id,name,subject_id,golden_mouse,genotype,date_of_birth,sex,modality,session_date,age,session_time,column,volume
0,V1 Deep Dive,3b85c659-20c8-438f-ab58-de1aac3b81cf,416296_2018-11-29_12-08-40_nwb_2025-08-08_16-2...,416296,False,Camk2a-tTA/wt;tetO-GCaMP6s/wt,2018-08-06,Female,"['Planar optical physiology', 'Behavior videos']",2018-11-29,115,12:08:40.014190,2,5
1,V1 Deep Dive,b98ce4a9-a66b-4b70-baa6-95ef451ae087,427836_2019-04-25_12-16-58_nwb_2025-08-08_16-2...,427836,False,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,"['Planar optical physiology', 'Behavior videos']",2019-04-25,199,12:16:58.240890,5,3
2,V1 Deep Dive,5cccd09c-4ae8-4d8e-ae92-23099d22bbe2,427836_2019-04-24_13-06-45_nwb_2025-08-08_16-2...,427836,False,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,"['Planar optical physiology', 'Behavior videos']",2019-04-24,198,13:06:45.257460,4,3
3,V1 Deep Dive,911d215f-dafa-4b39-86f7-b36336974da1,427836_2019-04-25_13-49-39_nwb_2025-08-08_16-2...,427836,False,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,"['Planar optical physiology', 'Behavior videos']",2019-04-25,199,13:49:39.163550,5,4
4,V1 Deep Dive,c918f874-d534-4341-b0dc-f1aba388af66,427836_2019-04-26_12-54-40_nwb_2025-08-08_16-2...,427836,False,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,"['Planar optical physiology', 'Behavior videos']",2019-04-26,200,12:54:40.332660,2,4


In [9]:
# Select golden mouse metadata 
goldenMouse_metadata = metadata[metadata.golden_mouse]
goldenMouse_metadata



,project_name,_id,name,subject_id,golden_mouse,genotype,date_of_birth,sex,modality,session_date,age,session_time,column,volume
35,V1 Deep Dive,8ceff8e1-d9ed-44b4-a051-315484744950,409828_2018-11-06_14-02-59_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-06,126,14:02:59.889330,2,1
36,V1 Deep Dive,d7cfeabb-99ff-435c-8aa5-f7340798f8ff,409828_2018-11-21_09-22-23_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-21,141,09:22:23.259970,4,1
37,V1 Deep Dive,e5397ea4-8462-4188-be4b-71931d9cfffd,409828_2018-11-20_10-42-45_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-20,140,10:42:45.603660,3,1
38,V1 Deep Dive,6498d9ee-9bf4-4980-a302-179ac06814c9,409828_2018-11-20_12-12-08_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-20,140,12:12:08.089640,3,2
39,V1 Deep Dive,83ab1835-f756-4830-9eba-db30de3290fc,409828_2018-11-28_10-54-56_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-28,148,10:54:56.809310,3,3
40,V1 Deep Dive,b9c86d99-7622-48c8-8144-899678d84317,409828_2018-11-26_11-16-25_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-26,146,11:16:25.373720,5,1
41,V1 Deep Dive,577a9e60-3b57-4515-bbc5-54ae5ff23a5c,409828_2018-11-27_12-29-05_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-27,147,12:29:05.453040,2,3
42,V1 Deep Dive,6ebe4b1a-a85b-44c8-985d-4b3f2ffa2db9,409828_2018-11-27_11-01-58_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-27,147,11:01:58.469640,2,2
43,V1 Deep Dive,274c5194-ffea-4ab3-ab2b-415eaad7faaf,409828_2018-11-21_10-56-07_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-11-21,141,10:56:07.266840,4,2
44,V1 Deep Dive,ea2f784b-e9ad-4724-974a-3c6047a30a79,409828_2018-12-03_14-25-24_nwb_2025-08-08_16-2...,409828,True,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-07-03,Male,"['Planar optical physiology', 'Behavior videos']",2018-12-03,153,14:25:24.632480,4,3


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<b> CAVE account setup</b>

<p>In order to manage server traffic, every user needs to create a CAVE account and download a user token to access CAVE's services programmatically. The CAVE infrastructure can be read about in <a href='https://doi.org/10.1038/s41592-024-02426-z'>more detail in the MICrONS Nature Package</a>. 

<b> A Google account (or Google-enabled account) is required to create a CAVE account.</b>

Run the cell below and follow the set up instructions; enter the token first, then enter 'y' when prompted

</div>

In [10]:
# Token Generation and Setup
CAVEclient.setup_token("https://global.em.brain.allentech.org")

# Initialize CAVEclient
client = CAVEclient(datastack_name="v1dd_public")

Visit https://global.em.brain.allentech.org/sticky_auth/settings/tokens and copy an existing token or create a new one.


Paste your auth token (input will be hidden):  ········
Set all of your datastacks to use this token and server ('Y', recommended), specify individual datastacks ('n'), or finish now ('exit'). (Y/n/exit)  y


All datastacks are configured to use this server address.
You will not need to specify a server address when initializing a client for configured datastacks in the future.
Setup complete!


In [14]:
client

CAVEclient<datastack_name=v1dd_public, server_address=https://global.em.brain.allentech.org>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Proofreading and data quality

Understanding this variablity in data quality is critical when interpretting electron microscopy reconstructions.

Automated segmentation of neuronal processes in dense EM imaging is challenging at the size of entire neurons, which can have millimeters of axons and dendrites. The automated segmentation algorithms used in the EM data for this project are not perfect, and so proofreading is necessary to obtain accurate reconstructions of a cell and confidence in the connectivity

In general, the **dendrites** of single-soma detected neurons are considered trustworthy, even without proofreading. However the **axons** require manual effort to both clean and extend. 

For details on the proofreading status see: 

* Proofreading page in the <a href=https://allenswdb.github.io/anatomy/microns-em/proofreading.html> SWDB databook</a>

<b> For this project, we treat all cells with at least `axon_partially_extended` as equally trustworth.</b> This may not be a safe assumption for all analysis, however, this proofreading method was chosen for consisitency with workshop codes.

<font size=3>Loading prepared variables: <code>dendrite_proof_root_ids</code> and <code>axon_proof_root_ids</code> </font>
    
</div>

In [13]:
# Loads cells with axon and dendrite proofreading
dendrite_proof_root_ids = np.load(pjoin(data_dir, f'proofread_dendrite_list_{mat_version}.npy'))
axon_proof_root_ids = np.load(pjoin(data_dir, f'proofread_axon_list_{mat_version}.npy'))

print(f'There are {len(dendrite_proof_root_ids)} cells with acceptable dendrites, and {len(axon_proof_root_ids)} cells with axon proofreading')
print(f'The number of cells with usable axons and dendrites is: {len(np.isin(axon_proof_root_ids, dendrite_proof_root_ids))}')

FileNotFoundError: [Errno 2] No such file or directory: '/data/409828_V1DD_GoldenMouse/proofread_dendrite_list_1196.npy'